# submission 데이터 전처리

In [1]:
import pandas as pd
import numpy as np
import gc

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen
from rdkit.Chem import QED
from sklearn.preprocessing import OneHotEncoder
import concurrent.futures
import warnings

# 오류 경고 무시하기
warnings.filterwarnings(action='ignore')

### data load

In [2]:
df = pd.read_csv('../leash-BELKA/test.csv') # data load
df.drop(['buildingblock1_smiles', 'buildingblock2_smiles', 'buildingblock3_smiles'], axis=1, inplace=True) # 필요없는 feature 삭제

In [3]:
df.shape

(1674896, 3)

### feature 생성

In [4]:
# ecfp 생성 함수
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

# atomic_num 생성 함수
def generate_atomic_num(chunk):
    print('start')
    atomic_num_li = []
    n = chunk.shape[0]
    print(n)

    for i in range(n):
        atomic_num_li.append(chunk['molecule'].tolist()[i].GetNumAtoms())

    return atomic_num_li

# 각 feature에 대한 함수 정의
make_features_dict = {'molecule': Chem.MolFromSmiles,
                      'ecfp': generate_ecfp,
                      'reactivity': rdMolDescriptors.CalcChi0n,
                      'molecular weight': Descriptors.MolWt,
                      'Steric strain': rdMolDescriptors.CalcNumRotatableBonds,
                      'LogP': Crippen.MolLogP,
                      'TPSA': rdMolDescriptors.CalcTPSA,
                      'NHBD': rdMolDescriptors.CalcNumHBD,
                      'NHBA': rdMolDescriptors.CalcNumRotatableBonds,
                      'planarity': rdMolDescriptors.CalcNumAromaticRings,
                      'PSA': rdMolDescriptors.CalcTPSA,
                      'QED': QED.qed,
                      'atomic_num': generate_atomic_num
                     }

features = make_features_dict.keys() # feature 리스트

# 쓰레드에서 수행할 함수
def process_chunk(chunk, rank):
    for feature in features:
        if feature == 'molecule':
            chunk[feature] = chunk['molecule_smiles'].apply(make_features_dict[feature])
        elif feature == 'atomic_num':
            chunk[feature] = generate_atomic_num(chunk)
        else :
            chunk[feature] = chunk['molecule'].apply(make_features_dict[feature])
    
    return (chunk, rank)

n =  418724 # 전체 데이터셋에서 나눠지는 데이터의 개수 -> 한 데이터셋 마다 418724개 저장
m = 4 # 만들어지는 그룹 개수

max_workers = 4 # 사용할 쓰레드 개수
CHUNK_SIZE = n // max_workers # 한 쓰레드에 들어갈 chunk 크기 -> 418724 // 4 = 104681

In [5]:
# 테스트 데이터를 418724개씩 나눠준다.
# 4개의 그룹 형성

split_df_li = []
for i in range(m):
    start = i * n
    end = (i+1) * n
    split_df_li.append(df.iloc[start:end])

cnt = 0 # 잘 작동하는지 확인


for split_df in split_df_li:
    print(cnt, 'START', end=' ') # 시작
    thread_result_li = [] # 각 쓰레드의 결과 저장 리스트

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [] # 쓰레드들의 집합
        rank = 0 # 쓰레드가 끝나는 시점이 다르기에 순서를 찾기위해 사용
        for start in range(0, n, CHUNK_SIZE):
            end = start + CHUNK_SIZE
            chunk = split_df.iloc[start:end] # 쓰레드에 들어갈 데이터 추출 -> 150000 / 6 = 25000
            future = executor.submit(process_chunk, chunk, rank) # 쓰레드에 할일 지정
            futures.append(future) # 쓰레드를 리스트에 추가
            rank += 1 
            
            del future
            gc.collect()

        for future in concurrent.futures.as_completed(futures):
            result = future.result() # 쓰레드에서 끝난 결과 반환
            thread_result_li.append(result) # 결과들을 하나의 리스트에 저장

            del result
            gc.collect()
        
        thread_result_li.sort(key= lambda x: x[1])
        print('sort complete')

    # 모든 결과를 하나의 dataframe으로 병합
    full_df_tmp = pd.concat([thread_result_li[i][0] for i in range(max_workers)], axis=0, ignore_index=True)
    ecfp_df_tmp = pd.DataFrame(full_df_tmp['ecfp'].to_list()) # ecfp를 dataframe 형태로 변환후 더해준다.
    full_df_tmp.drop(['molecule_smiles', 'molecule', 'ecfp'], axis=1, inplace=True) # 필요없는 데이터 삭제 -> molecule, ecfp
    full_df = pd.concat([full_df_tmp, ecfp_df_tmp], axis=1) # ecfp 추가
    full_df.to_csv(f'./submission_data2/{cnt}.csv', index=False) # 데이터 저장

    del thread_result_li
    del futures
    del full_df_tmp
    del ecfp_df_tmp
    del full_df
    gc.collect()

    cnt += 1
    print('END')

del df
del split_df_li
gc.collect()

0 START start
104681
start
104681
start
104681
start
104681
sort complete
END
1 START start
104681
start
104681
start
104681
start
104681
sort complete
END
2 START start
104681
start
104681
start
104681
start
104681
sort complete
END
3 START start
104681
start
104681
start
104681
start
104681
sort complete
END


0